In [1]:
import cv2

In [2]:
cap = cv2.VideoCapture(0)

In [3]:
# cap.set(cv2.CAP_PROP_ZOOM, 200)

In [4]:
if not cap.isOpened():
    print("Camera not found")
    exit()

cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
print(f"{int(width)}x{int(height)}")
print(cap.get(cv2.CAP_PROP_ZOOM))

1280x480
100.0


In [ ]:
# StereoBM 객체 생성 (루프 외부에서 한 번만 생성)
# numDisparities: 시차 탐색 범위 (16의 배수여야 함, 예: 16, 32, ... 96 ...)
# blockSize: 매칭 블록 크기 (홀수여야 함, 예: 5, 7, ... 21 ...)
stereo = cv2.StereoBM_create(numDisparities=32, blockSize=21)

while True:
    ret, frame = cap.read()

    if not ret:
        print("Do not read frame")
        break

    h, w  = frame.shape[:2]

    # 가로 분할 (좌/우 카메라 이미지 분리)
    half_width = w // 2
    left_camera = frame[0:h, 0:half_width]
    right_camera = frame[0:h, half_width:w]

    # 깊이 계산을 위해 그레이스케일로 변환
    left_gray = cv2.cvtColor(left_camera, cv2.COLOR_BGR2GRAY)
    right_gray = cv2.cvtColor(right_camera, cv2.COLOR_BGR2GRAY)

    # Disparity(시차) 계산
    # 결과는 16비트 고정 소수점 형식이므로 바로 시각화하기 어렵습니다.
    disparity = stereo.compute(left_gray, right_gray)

    # 시각화를 위해 정규화 (0~255 범위의 8비트 이미지로 변환)
    norm_disparity = cv2.normalize(disparity, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    
    # 깊이 맵에 컬러맵 적용 (JET 컬러맵 사용)
    depth_map = cv2.applyColorMap(norm_disparity, cv2.COLORMAP_JET)

    # 결과 합치기 (왼쪽 원본 영상과 깊이 맵을 나란히 표시)
    combined_frame = cv2.hconcat([left_camera, depth_map])

    cv2.imshow("View", combined_frame)
    
    if cv2.waitKey(1) & 0xff == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

: 